In [9]:
%pylab inline
import platform
import IPython
import sklearn as sk
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

print ('Python version:', platform.python_version())
print ('IPython version:', IPython.__version__)
print ('numpy version:', np.__version__)
print ('scikit-learn version:', sk.__version__)
print ('matplotlib version:', matplotlib.__version__)

Populating the interactive namespace from numpy and matplotlib
Python version: 3.5.2
IPython version: 4.0.1
numpy version: 1.13.1
scikit-learn version: 0.18.2
matplotlib version: 1.5.0


In [15]:
#Ignore warnings 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

We will compare several regression methods by using the same
dataset. We will try to predict the price of a house as a function of its attributes.
As the dataset, we will use the Boston house-prices dataset, which includes 506
instances, representing houses in the suburbs of Boston by 14 features, one of them
(the median value of owner-occupied homes) being the target class (for a detailed
reference, see http://archive.ics.uci.edu/ml/datasets/Housing). Each
attribute in this dataset is real-valued.

In [16]:
from sklearn.datasets import load_boston
boston = load_boston()
print (boston.data.shape)
print (boston.feature_names)
print (np.max(boston.target), np.min(boston.target), np.mean(boston.target))

(506, 13)
['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
50.0 5.0 22.5328063241


In [17]:
print (boston.DESCR)

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

As usual, we start slicing our learning set into training and testing datasets, and
normalizing the data:

In [18]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.25, random_state=33)
from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler().fit(X_train)
scalery = StandardScaler().fit(y_train)
X_train = scalerX.transform(X_train)
y_train = scalery.transform(y_train)
X_test = scalerX.transform(X_test)
y_test = scalery.transform(y_test)

Before looking at our best classifier, let's define how we will compare our results.
Since we want to preserve our testing set for evaluating the performance of the final
classifier, we should find a way to select the best model while avoiding overfitting.
We already know the answer: cross-validation. Regression poses an additional
problem: how should we evaluate our results? Accuracy is not a good idea, since
we are predicting real values, it is almost impossible for us to predict exactly the
final value. There are several measures that can be used (you can look at the list of
functions under sklearn.metrics module). The most common is the R2 score, or
coefficient of determination that measures the proportion of the outcomes variation
explained by the model, and is the default score function for regression methods in
scikit-learn. This score reaches its maximum value of 1 when the model perfectly
predicts all the test target values. Using this measure, we will build a function that
trains a model and evaluates its performance using five-fold cross-validation and the
coefficient of determination.

In [20]:
from sklearn.cross_validation import *

def train_and_evaluate(clf, X_train, y_train):
    clf.fit(X_train, y_train)
    print ("Coefficient of determination on training set:",clf.score(X_train, y_train))
    # create a k-fold cross validation iterator of k=5 folds
    cv = KFold(X_train.shape[0], 5, shuffle=True, random_state=33)
    scores = cross_val_score(clf, X_train, y_train, cv=cv)
    print ("Average coefficient of determination using 5-fold crossvalidation:",np.mean(scores))

# First try – a linear model

The question that linear models try to answer is which hyperplane in the
14-dimensional space created by our learning features (including the target value)
is located closer to them. After this hyperplane is found, prediction reduces to
calculate the projection on the hyperplane of the new point, and returning the target
value coordinate. Think of our first example in Chapter 1, Machine Learning – A Gentle
Introduction, where we wanted to find a line separating our training instances.
We could have used that line to predict the second learning attribute as a function
of the first one, that is, linear regression.

But, what do we mean by closer? The usual measure is least squares: calculate the
distance of each instance to the hyperplane, square it (to avoid sign problems), and
sum them. The hyperplane whose sum is smaller is the least squares estimator (the
hyperplane in the case if two dimensions are just a line).

Since we don't know how our data fits (it is difficult to print a 14-dimension
scatter plot!), we will start with a linear model called SGDRegressor, which tries to
minimize squared loss.

In [21]:
from sklearn import linear_model
clf_sgd = linear_model.SGDRegressor(loss='squared_loss', penalty=None, random_state=42)
train_and_evaluate(clf_sgd,X_train,y_train)

Coefficient of determination on training set: 0.743617732983
Average coefficient of determination using 5-fold crossvalidation: 0.710809853468


We can print the hyperplane coefficients our method has calculated, which is
as follows:

In [22]:
print (clf_sgd.coef_)

[-0.08527595  0.06706144 -0.05032898  0.10874804 -0.07755151  0.38961893
 -0.02485839 -0.20990016  0.08491659 -0.05495108 -0.19854006  0.06126093
 -0.37817963]
